In [1]:
import networkx as nx
import matplotlib.pyplot as plt

from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit_aer import Aer
from qiskit.circuit import Parameter
from qiskit.visualization import plot_histogram

import numpy as np
from scipy.optimize import minimize

In [2]:
# dij需求i到设施j的成本
# d = [[1, 2], [1, 2]]
# n = 2   # 两个设施点
# m = 2   # 两个需求点
d = [[1, 2], [3, 4], [5, 6]]
n = 2   # 两个设施点
m = 3   # 三个需求点
num_qubits = n + 2 * n * m

# gi设施i的建设成本
g = [2, 1]


In [3]:
penalty = 40
depth = 5
params = np.ones(depth * 2)

In [4]:
GateX = np.array([[0, 1],[1, 0]])
GateY = np.array([[0, -1j],[1j, 0]])
GateZ = np.array([[1, 0],[0, -1]])

In [5]:
# dtype=np.complex128
def add_in_target(num_qubits, target_qubit, gate=np.array([[1, 0],[0, -1]])):
    H = np.eye(2 ** (target_qubit))
    H = np.kron(H, gate)
    H = np.kron(H, np.eye(2 ** (num_qubits - 1 - target_qubit)))
    return H


def generate_Hp(n, m, d, g):
    # 初始化 Hp 矩阵为零矩阵
    # print(num_qubits)
    Hp = np.zeros((2**num_qubits, 2**num_qubits))
    for i in range(m):
        for j in range(n):
            Hp += d[i][j] * (add_in_target(num_qubits, n * (1 + i) + j) - np.eye(2**num_qubits)) / 2
    
    for j in range(n):
        Hp +=  g[j] * (add_in_target(num_qubits, j)- np.eye(2**num_qubits)) / 2

    # for i in range(m):
    #     Ht = np.zeros((2**num_qubits, 2**num_qubits))
    #     for j in range(n):
    #         Ht += (np.eye(2**num_qubits) - add_in_target(num_qubits, n * (1 + i) + j)) / 2
    #     Ht -= np.eye(2**num_qubits)
    #     Hp += -penalty * Ht @ Ht
    
    for i in range(m):
        for j in range(n):
            Ht = (np.eye(2**num_qubits) - add_in_target(num_qubits, n * (1 + i) + j)) / 2 + (np.eye(2**num_qubits) - add_in_target(num_qubits, n * (1 + m + i) + j)) / 2 + (np.eye(2**num_qubits) - add_in_target(num_qubits, j)) / 2
            Hp += -penalty * Ht @ Ht

    return Hp

In [6]:
def gnrt_Hd(n, m):
  mn = m * n
  Hd = np.zeros((2**mn, 2**mn)).astype(np.complex128)
  for i in range(mn):
    j = (i + 1) % mn
    Hd += (add_in_target(mn, i, GateX) @ add_in_target(mn, j, GateX) + add_in_target(mn, i, GateY) @ add_in_target(mn, j, GateY))
  return -Hd


In [7]:
from scipy.linalg import expm
def build_circ(n, m, d, g, params):
  qc = QuantumCircuit(num_qubits)
  beta = params[:depth]
  gamma = params[depth:]
  for i in range(m * n):
    qc.h(i)
  for i in range(m * n, m * n * 2, 2):
    qc.x(i)
  for i in range(m * n * 2, num_qubits):
    qc.h(i)
  for dp in range(depth):
    qc.unitary(expm(-1j * gamma[dp] * generate_Hp(n, m, d, g)), range(num_qubits)) # transpile
    # 
    for i in range(m * n):
      qc.rx(beta[dp], i)
    qc.unitary(expm(-1j * beta[dp] * gnrt_Hd(n, m)), range(m * n , 2 * m * n))
    for i in range(2 * m * n, num_qubits):
      qc.rx(beta[dp], i)
  qc.measure_all()
  return qc

In [8]:
print(build_circ(n, m, d, g, np.full(depth * 2, np.pi/4)))

         ┌───┐┌───────────┐┌─────────┐ ┌───────────┐┌─────────┐ ┌───────────┐»
    q_0: ┤ H ├┤0          ├┤ Rx(π/4) ├─┤0          ├┤ Rx(π/4) ├─┤0          ├»
         ├───┤│           │├─────────┤ │           │├─────────┤ │           │»
    q_1: ┤ H ├┤1          ├┤ Rx(π/4) ├─┤1          ├┤ Rx(π/4) ├─┤1          ├»
         ├───┤│           │├─────────┤ │           │├─────────┤ │           │»
    q_2: ┤ H ├┤2          ├┤ Rx(π/4) ├─┤2          ├┤ Rx(π/4) ├─┤2          ├»
         ├───┤│           │├─────────┤ │           │├─────────┤ │           │»
    q_3: ┤ H ├┤3          ├┤ Rx(π/4) ├─┤3          ├┤ Rx(π/4) ├─┤3          ├»
         ├───┤│           │├─────────┤ │           │├─────────┤ │           │»
    q_4: ┤ H ├┤4          ├┤ Rx(π/4) ├─┤4          ├┤ Rx(π/4) ├─┤4          ├»
         ├───┤│           │├─────────┤ │           │├─────────┤ │           │»
    q_5: ┤ H ├┤5          ├┤ Rx(π/4) ├─┤5          ├┤ Rx(π/4) ├─┤5          ├»
         ├───┤│           │├─────────┴┐│           │

In [9]:
def cost_function(x):
  num = [int(char) for char in x]
  C = 0
  for i in range(m):
    for j in range(n):
      C += d[i][j] * num[n * (1 + i) + j]
      
  for j in range(n):
    C += g[j] * num[j]

  # for i in range(m):
  #   t = 0
  #   for j in range(n):
  #     t += num[n * (1 + i) + j]
  #   C += penalty * (t - 1)**2

  for i in range(m):
    for j in range(n):
      C += penalty * (num[n * (1 + i) + j] + num[n * (1 + m + i) + j] - num[j]) ** 2
  return C

def compute_expectation(counts):
  EV = 0
  total_count = 0
  for x, count in counts.items():
    C = cost_function(x)
    EV += C*count
    total_count += count

  return EV/total_count


def expectation_from_sample(shots = 2000):
  backend = Aer.get_backend('qasm_simulator')
  backend.shots = shots

  def execute_circ(theta):
    qc = build_circ(n, m, d, g, theta)
    counts = backend.run(qc, seed_simulator=10, shots=shots).result().get_counts()
    return compute_expectation(counts)
  
  return execute_circ

In [10]:
from numpy.lib.utils import source
from scipy.optimize import minimize
import numpy as np
# 初始化迭代计数器
iteration_count = 0
def test(pen, dep, par):
  global penalty, depth, params, iteration_count
  iteration_count = 0
  penalty = pen
  depth = dep
  params = par
  expectation = expectation_from_sample()
  def callback(x):
      global iteration_count
      iteration_count += 1
      if iteration_count % 10 == 0:
          print(f"Iteration {iteration_count}, Result: {expectation(x)}")
  # 设定最大迭代次数
  max_iterations = 1000

  # 使用 COBYLA 方法进行最小化，并设置 callback 函数
  res = minimize(expectation, params, method='COBYLA', options={'maxiter': max_iterations}, callback=callback)
  # 输出最终结果
  print("Final Result:", res)
  backend = Aer.get_backend('aer_simulator')
  backend.shots = 100000

  shots=100000
  qc_res = build_circ(n, m, d, g, params=res.x)

  counts = backend.run(qc_res, seed_simulator=10, shots = shots).result().get_counts()
  # plot_histogram(counts)
  sorted_counts = sorted(counts, key=counts.get, reverse=True)
  print("\n----------------- Full result ---------------------")
  print("selection\t\tprobability\tvalue")
  print("---------------------------------------------------")
  for x in sorted_counts[:20]:
    print(x, "{:.1f}%".format(counts[x] / shots * 100), cost_function(x))

In [11]:
test(40, 1, np.full(1 * 2, np.pi))

Iteration 10, Result: 246.309
Iteration 20, Result: 270.882
Iteration 30, Result: 158.754
Final Result:  message: Optimization terminated successfully.
 success: True
  status: 1
     fun: 159.407
       x: [ 4.127e+00  2.504e+00]
    nfev: 33
   maxcv: 0.0

----------------- Full result ---------------------
selection		probability	value
---------------------------------------------------
10101010000000 6.4% 11
00101010000000 3.2% 129
10100011000000 2.4% 94
10111000000000 2.4% 88
10001110000000 2.3% 94
11101010000000 1.7% 132
10101010001000 1.6% 51
10101010100000 1.5% 51
10101010000010 1.5% 51
00111000000000 1.2% 126
00001110000000 1.2% 132
00100011000000 1.2% 132
10101010000100 1.0% 51
10101010010000 0.9% 51
00101010100000 0.9% 249
00101010000010 0.9% 249
00101010001000 0.9% 249
10101010000001 0.9% 51
01101010000000 0.9% 250
10000111000000 0.7% 177


In [12]:
test(40, 2, np.full(2 * 2, np.pi))

Iteration 10, Result: 250.8535
Iteration 20, Result: 259.8615
Iteration 30, Result: 141.1735
Iteration 40, Result: 138.32
Iteration 50, Result: 137.903
Final Result:  message: Optimization terminated successfully.
 success: True
  status: 1
     fun: 137.903
       x: [ 4.030e+00  3.365e+00  4.094e+00  3.073e+00]
    nfev: 51
   maxcv: 0.0

----------------- Full result ---------------------
selection		probability	value
---------------------------------------------------
10101010000000 4.2% 11
10001110000000 2.2% 94
10100011000000 2.1% 94
10111000000000 2.1% 88
00101010000000 1.7% 129
10101010100000 1.6% 51
10101010001000 1.4% 51
10101010000010 1.4% 51
00100011000000 1.0% 132
00001110000000 0.9% 132
00111000000000 0.9% 126
10001011000000 0.9% 96
10001110100000 0.8% 54
10100011001000 0.8% 54
10111000000010 0.8% 48
10001110001000 0.7% 134
10100110000000 0.7% 92
10100011100000 0.7% 134
10111000100000 0.7% 128
10100011000010 0.7% 134


In [13]:
test(40, 3, np.full(3 * 2, np.pi/3))

Iteration 10, Result: 253.7035
Iteration 20, Result: 263.871
Iteration 30, Result: 250.98
Iteration 40, Result: 202.712
Iteration 50, Result: 175.266
Iteration 60, Result: 170.385
Final Result:  message: Optimization terminated successfully.
 success: True
  status: 1
     fun: 171.2065
       x: [ 2.125e+00  9.854e-01  2.082e+00  8.191e-01  2.047e+00
            2.170e+00]
    nfev: 68
   maxcv: 0.0

----------------- Full result ---------------------
selection		probability	value
---------------------------------------------------
10101010000001 3.5% 51
10101010010000 3.5% 51
10101010000100 3.5% 51
11101010010101 2.4% 12
10101010010101 2.0% 131
11101010000101 1.9% 52
11101010010100 1.8% 52
11101010010001 1.7% 52
10101010000000 1.2% 11
10001110010000 1.1% 134
10001110000001 1.1% 134
10100011010000 1.1% 134
10100011000100 1.1% 134
10111000000100 1.0% 128
10111000000001 1.0% 128
11101010010000 1.0% 92
11001110010101 1.0% 95
11100011010101 0.9% 95
11101010000001 0.8% 92
11111000010101 0.8

In [14]:
test(40, 5, np.full(5 * 2, np.pi/4))

Iteration 10, Result: 205.624
Iteration 20, Result: 252.8135
Iteration 30, Result: 241.632
Iteration 40, Result: 180.3425
Iteration 50, Result: 192.272
Iteration 60, Result: 167.5205
Iteration 70, Result: 162.0465
Iteration 80, Result: 158.8765
Iteration 90, Result: 159.2235
Iteration 100, Result: 159.223
Final Result:  message: Optimization terminated successfully.
 success: True
  status: 1
     fun: 155.8175
       x: [ 3.568e-01  1.792e+00  1.778e+00  1.040e+00  7.857e-01
            1.016e+00  6.991e-01  1.787e+00  7.839e-01  1.784e+00]
    nfev: 103
   maxcv: 0.0

----------------- Full result ---------------------
selection		probability	value
---------------------------------------------------
11101010000000 3.0% 132
10101010000000 1.9% 11
10110010000000 1.4% 90
11110010000000 1.3% 131
00111000000000 1.2% 126
10101100000000 1.2% 90
01111000000000 1.1% 167
11101100000000 1.0% 131
11001011000000 1.0% 137
11001110000000 0.9% 135
11111000000000 0.8% 129
11110010001000 0.7% 91
111010